# Data Analysis


To develop a classifier that can decide which one of two prompts will be prefered by a user we have to identify what data is relevant for a user to like or dislike a text. This data analysis aims to sort out the relevant data for our task.

We have the following assumptions to solve the task:
- The liking of a text is not random but follows certain patterns
- These patterns can be identified through data analysis
- The patterns are related to measurable text characteristics
- Users have consistent preferences that can be learned
- The preferences can be generalized across different texts


Possibly relevant data: 
1. Amount of nouns in response
2. Amount of adjectives in response
3. sentence-sentiment
4. amount of words per response
5. amount of (., ,, -, !, etc.) in response
6. amount of letters per response

In [2]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler


In [14]:
# Import nouns and adjectives
from nltk.corpus import wordnet as wn
import nltk

# Ensure WordNet is downloaded
nltk.download('wordnet')

# Function to extract all nouns
def get_nouns():
    nouns = set()
    for synset in wn.all_synsets(pos=wn.NOUN):
        for lemma in synset.lemmas():
            name = lemma.name()
            # Replace underscores with empty string to concatenate words
            name = name.replace('_', '')
            nouns.add(name)
    return nouns

# Function to extract all adjectives
def get_adjectives():
    adjectives = set()
    for synset in wn.all_synsets(pos=wn.ADJ):
        for lemma in synset.lemmas():
            adjectives.add(lemma.name())
    return adjectives

# Get nouns and adjectives
nouns = get_nouns()
adjectives = get_adjectives()

print("Nouns: ", list(nouns)[:10])
print("Adjectives: ", list(adjectives)[:10])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\siran\AppData\Roaming\nltk_data...


In [4]:
# Load the different datasets
df_train = pd.read_csv('../Data/LLM Classification Finetuning/train.csv')
df_test = pd.read_csv('../Data/LLM Classification Finetuning/test.csv')

# Display basic information about the datasets
print("Training dataset shape:", df_train.shape)
print("Test dataset shape:", df_test.shape) 
print("Dataset columns:", df_train.columns)


Training dataset shape: (57477, 9)
Test dataset shape: (3, 4)
Dataset columns: Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')


## 1. Amount of nouns in response

In [32]:
def count_nouns(sentence):
    words = sentence.split(" ")
    count = 0
    for word in words:
        if word.lower() in nouns:
            count += 1
    return count

    
def count_nouns_rows(row):
    a_count = count_nouns(row["response_a"])
    b_count = count_nouns(row["response_b"])
    return a_count, b_count


In [35]:
# The apply function returns a Series/array, need to convert to separate columns
counts = df_train.apply(count_nouns_rows, axis=1)
df_train["count_a"] = [x[0] for x in counts]
df_train["count_b"] = [x[1] for x in counts]

In [37]:
print(df_train[:3])

      id             model_a         model_b  \
0  30192  gpt-4-1106-preview      gpt-4-0613   
1  53567           koala-13b      gpt-4-0613   
2  65089  gpt-3.5-turbo-0613  mistral-medium   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   
1  ["What is the difference between marriage lice...   
2  ["explain function calling. how would you call...   

                                          response_a  \
0  ["The question of whether it is morally right ...   
1  ["A marriage license is a legal document that ...   
2  ["Function calling is the process of invoking ...   

                                          response_b  winner_model_a  \
0  ["As an AI, I don't have personal beliefs or o...               1   
1  ["A marriage license and a marriage certificat...               0   
2  ["Function calling is the process of invoking ...               0   

   winner_model_b  winner_tie  count_a  count_b  
0               0 

In [ ]:
# TODO model: amount of nouns ; answer prefered